# Predicting Special Team Play Result and Player Recommendation Using Knowledge Graph

 
1.	Introduction

American Football is one of America's most popular strategic sports, played by two teams of eleven players on a rectangular field with goalposts at each end. Each team in American football consists of 3 different units: offensive team, defensive team, and special team, which have their respective functions in the game. Unlike offence or defence, special teams are responsible for scoring points and protecting points and field positions. According to Will Hewlett, special teams are one of the most overlooked aspects of football [1]. Although Many people might underestimate the impact of special teams because special team units are only on the field for about 20% of the play in most games, special team play can mean the difference between winning and losing[2]. The performance of a special team alone will not win a football game, but it can make a significant impact on the result, with a difference of more than a goal between the best and worst punt units.
As a collective term, Special teams are made up of several similar, but ultimately different, units: punt, punt return, kick-off, kick-off return, field goal/PAT, and field goal/PAT block[3]. Each unit has its specialization and specific strategies to get a score or good field position. In general, there are 4 types of games played by special teams, namely Kick off, Punt, Field Goal and Extra Point. Unlike Field Goal and Extra Point, Kickoff and Punt can change the game direction; besides aiming to secure the field position, this type of game also has its own risks and advantages. For example, if the kicking team do a Kick-off and the return team manages to catch the ball and bring it to the end of the opponent's zone, this will give the opponent a significant advantage. Therefore, selecting the play type result strategy from the type of game will determine the game's outcome. Because of the importance of the special team play type, especially Kick-off and Punt's performances, our goal is to predict the special team play type result to improve the team winning strategy. 
Objectives :
The long term goal of our project is to predict the special team play type result (score or not score) to improve the team winning strategy. The objective of our projects such as :
1.	To predict the special team result ( score or not score) based on a related dataset provided by Kaggle.
2.	To compare and evaluate the performance of the selected supervised learning algorithm.
3.	To provide final recommendations to those in need regarding a good strategy in improving the special teams' performance.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import networkx as nx
import matplotlib.pyplot as plt # for plot
from matplotlib.pyplot import figure

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This notebook is intended to be a study of the NFL Big Data Bowl data, as well as some general engineering and building classification models of interest.
The major focus here is feature engineering, which uses four of the NFL's special teams datasets to develop a model that can predict whether a punt or a kickoff will result in a touchdown or not.
Another goal of this notebook is to explore more about the relation between players using Knowledge Graph and in the end, will result a recommender system
So, this notebook consist of 2 part :
Part 1: Prediction Model
Part 2: Knowledge Graph

# **PART I. PREDICTION MODEL**

Predicting Play Type Result ( Score or not Score)

## LOADING DATASET
Load Dataset into dataframe

In [ ]:
df= pd.read_csv("../input/nfl-big-data-bowl-2022/PFFScoutingData.csv")
df2 = pd.read_csv("../input/nfl-big-data-bowl-2022/games.csv")
df3 = pd.read_csv("../input/nfl-big-data-bowl-2022/players.csv")
df4 = pd.read_csv("../input/nfl-big-data-bowl-2022/plays.csv")
df5 = pd.read_csv("../input/nfl-big-data-bowl-2022/tracking2018.csv")
#df5_2 = pd.read_csv("../input/nfl-big-data-bowl-2022/tracking2019.csv")
#df5_3 = pd.read_csv("../input/nfl-big-data-bowl-2022/tracking2020.csv")

## **DATA PREPOCESSING**

In [ ]:
#making dataframe using only related feature that we want to use
plays = df4[['gameId','playId','possessionTeam','specialTeamsPlayType','specialTeamsResult','kickerId','returnerId','kickBlockerId','penaltyCodes','penaltyYards','preSnapHomeScore','preSnapVisitorScore','passResult','kickLength','kickReturnYardage','playResult']]
games = df2[['gameId','homeTeamAbbr','visitorTeamAbbr']]
players = df3[['nflId','collegeName']]
scout = df[['gameId','playId','snapDetail','operationTime','hangTime','kickType','kickDirectionIntended','kickDirectionActual','returnDirectionIntended','returnDirectionActual','missedTackler','assistTackler','tackler','kickoffReturnFormation','gunners','puntRushers','specialTeamsSafeties','vises','kickContactType']]

#track2018 = df5[['s','a','dis','nflId','gameId','playId']]
#track2019 = df5_2[['s','a','dis','nflId','gameId','playId']]
#track2020 = ddf5_3[['s','a','dis','nflId','gameId','playId','jerseyNumber','team']]


### 1. Merge Plays and Scout dataset

In [ ]:
#merging plays dataset and scout dataset into play_scout
play_scout = pd.merge(plays, scout, how='left', on=['playId','gameId'])

In [ ]:
#separate multiple user in dataset using explode technique

play_scout = play_scout.assign(returnerId=play_scout['returnerId'].str.split(';')).explode('returnerId')
play_scout = play_scout.assign(penaltyCodes=play_scout['penaltyCodes'].str.split(';')).explode('penaltyCodes')
play_scout = play_scout.assign(missedTackler=play_scout['missedTackler'].str.split(';')).explode('missedTackler')
play_scout = play_scout.assign(assistTackler=play_scout['assistTackler'].str.split(';')).explode('assistTackler')
play_scout = play_scout.assign(gunners=play_scout['gunners'].str.split(';')).explode('gunners')
play_scout = play_scout.assign(puntRushers=play_scout['puntRushers'].str.split(';')).explode('puntRushers')
play_scout = play_scout.assign(specialTeamsSafeties=play_scout['specialTeamsSafeties'].str.split(';')).explode('specialTeamsSafeties')
play_scout = play_scout.assign(vises=play_scout['vises'].str.split(';')).explode('vises')

In [ ]:
#add new coulumn 'home team' of returner team into first dataframe by gameId
play_scout['homeTeam'] =play_scout.gameId.map(games.set_index('gameId')['homeTeamAbbr'].to_dict())

### 2. Merge Plays_scout and games dataset

In [ ]:
#merging dataframe play_scout and games into play_scout
play_scout=play_scout.merge(games[['gameId','homeTeamAbbr','visitorTeamAbbr']],how='left',left_on=['gameId','possessionTeam'],right_on=['gameId','homeTeamAbbr']).drop('homeTeamAbbr', axis='columns').fillna(games['homeTeamAbbr'])

In [ ]:
#define visitor team and add new coulumn 'visitor team'
for i in range(len(play_scout)):
  if (play_scout.loc[i, 'possessionTeam'] != play_scout.loc[i, 'homeTeam']) :
        play_scout.loc[i, 'visitorTeamAbbr'] = play_scout.loc[i, 'homeTeam']

In [ ]:
#rename visitor team to returner team
play_scout = play_scout.rename(columns={"visitorTeamAbbr": "returnerTeam"})

In [ ]:
play_scout.head()

#### In this part, I will generate new feature named "eventScore", 

In [ ]:
# calcalute home and visitor current score
for x in range(len(play_scout)-1):
  if play_scout.loc[x, 'preSnapHomeScore'] != play_scout.loc[x+1, 'preSnapHomeScore']:
    play_scout.loc[x, 'homeCurrentScore'] = (play_scout.loc[x+1, 'preSnapHomeScore']-play_scout.loc[x, 'preSnapHomeScore'])
  else:
    play_scout.loc[x, 'homeCurrentScore'] = 0
  if play_scout.loc[x, 'preSnapVisitorScore'] != play_scout.loc[x+1, 'preSnapVisitorScore']:
    play_scout.loc[x, 'visitorCurrentScore'] = (play_scout.loc[x+1, 'preSnapVisitorScore']-play_scout.loc[x, 'preSnapVisitorScore'])
  else:
    play_scout.loc[x, 'visitorCurrentScore'] = 0

In [ ]:
#calculate event score
play_scout['eventScore']= play_scout['homeCurrentScore']+play_scout['visitorCurrentScore']

In [ ]:
#normalized home and visitor score
for x in range(len(play_scout)-1):
  if play_scout.loc[x, 'homeCurrentScore']<0: 
    play_scout.loc[x, 'homeCurrentScore']=0
  if play_scout.loc[x, 'visitorCurrentScore']<0: 
    play_scout.loc[x, 'visitorCurrentScore']=0

In [ ]:
#define score or not score
for x in range(len(play_scout)-1):
  if play_scout.loc[x, 'eventScore'] > 0: #not score
    play_scout.loc[x, 'eventScore'] = 1
  else:
    play_scout.loc[x, 'eventScore'] = 0 #not score

### 3. Merge Plays_scout and players dataset

In [ ]:
#merging play_scout dataframa and players dataframe into play_scout for define kicker college
play_scout = pd.merge(play_scout, players, how='left', left_on=['kickerId'],right_on=['nflId'])

In [ ]:
del play_scout['nflId']

In [ ]:
#rename
play_scout = play_scout.rename(columns={"collegeName": "kickerCollege"})

In [ ]:
#change datatype for returner id
#play_scout['returnerId'] = play_scout['returnerId'].astype(np.float).astype("Int64")
play_scout['returnerId'] = play_scout['returnerId'].astype(np.float)

In [ ]:
#merging play_scout dataframa and players dataframe into play_scout for define returner college
play_scout = pd.merge(play_scout, players, how='left', left_on=['returnerId'],right_on=['nflId'])

In [ ]:
del play_scout['nflId']

In [ ]:
play_scout = play_scout.rename(columns={"collegeName": "returnerCollege"})

In [ ]:
play_scout.shape

In [ ]:
play_scout.head(1)

In [ ]:
play_scout.dtypes

In [ ]:
#define kick and return accuracy
for x in range(len(play_scout)-1):
  if play_scout.loc[x, 'kickDirectionIntended'] == play_scout.loc[x, 'kickDirectionActual']:
    play_scout.loc[x, 'kickAccuracy'] = 1
  else:
    play_scout.loc[x, 'kickAccuracy'] = 0
  if play_scout.loc[x, 'returnDirectionIntended'] == play_scout.loc[x, 'returnDirectionActual']:
    play_scout.loc[x, 'returnAccuracy'] = 1
  else:
    play_scout.loc[x, 'returnAccuracy']  = 0

### 4. Merge track and play_scout
#### Preparation

In [ ]:
#laoding data track
track = df5[['gameId','playId','nflId','jerseyNumber','team']]

In [ ]:
track.shape

In [ ]:
track.dropna()

In [ ]:
track = track.drop_duplicates()

In [ ]:
track = track.reset_index(drop=True)

In [ ]:
track.head()

In [ ]:
#add new coulumn "hometeam"
track['homeTeam'] =track.gameId.map(games.set_index('gameId')['homeTeamAbbr'].to_dict())

In [ ]:
#add new coulumn "awayteam"
track['awayTeam'] =track.gameId.map(games.set_index('gameId')['visitorTeamAbbr'].to_dict())

In [ ]:
#define team name for each player based on home team and visitor team
for x in range(len(track)-1):
  if track.loc[x, 'team'] == 'home':
    track.loc[x, 'teamName'] = track.loc[x, 'homeTeam']
  else:
    track.loc[x, 'teamName'] = track.loc[x, 'awayTeam']

In [ ]:
del track['team']

In [ ]:
del track['homeTeam']
del track['awayTeam']

In [ ]:
play_scout.head()

In [ ]:
# split data from  missed Tackler to gather Jersey number and Team Name
play_scout['missedTackler'] = play_scout['missedTackler'].str.replace(" ","")
play_scout['jersyMT']=play_scout['missedTackler'].str[3:]
play_scout['missedTackler'] = play_scout['missedTackler'].str[:3]


In [ ]:
# split data from  Assist Tackler to gather Jersey number and Team Name
play_scout['assistTackler'] = play_scout['assistTackler'].str.replace(" ","")
play_scout['jersyAT']=play_scout['assistTackler'].str[3:]
play_scout['assistTackler'] = play_scout['assistTackler'].str[:3]

In [ ]:
# split data from  Tackler to gather Jersey number and Team Name
play_scout['tackler'] = play_scout['tackler'].str.replace(" ","")
play_scout['jersyT']=play_scout['tackler'].str[3:]
play_scout['tackler'] = play_scout['tackler'].str[:3]



In [ ]:
# split data from  Gunners to gather Jersey number and Team Name
play_scout['gunners'] = play_scout['gunners'].str.replace(" ","")
play_scout['jersyG']=play_scout['gunners'].str[3:]
play_scout['gunners'] = play_scout['gunners'].str[:3]



In [ ]:
# split data from  puntRushers to gather Jersey number and Team Name
play_scout['puntRushers'] = play_scout['puntRushers'].str.replace(" ","")
play_scout['jersyPR']=play_scout['puntRushers'].str[3:]
play_scout['puntRushers'] = play_scout['puntRushers'].str[:3]

In [ ]:
# split data from  specialTeamsSafeties to gather Jersey number and Team Name
play_scout['specialTeamsSafeties'] = play_scout['specialTeamsSafeties'].str.replace(" ","")
play_scout['jersySTS']=play_scout['specialTeamsSafeties'].str[3:]
play_scout['specialTeamsSafeties'] = play_scout['specialTeamsSafeties'].str[:3]

In [ ]:
# split data from  vises to gather Jersey number and Team Name
play_scout['vises'] = play_scout['vises'].str.replace(" ","")
play_scout['jersyV']=play_scout['vises'].str[3:]
play_scout['vises'] = play_scout['vises'].str[:3]

In [ ]:
pd.set_option('display.max_columns', None)
play_scout.head()

In [ ]:
track.dtypes

In [ ]:
play_scout.dtypes

In [ ]:
#change data type to float
play_scout['jersyMT'] = play_scout['jersyMT'].astype(np.float)
play_scout['jersyAT'] = play_scout['jersyAT'].astype(np.float)
play_scout['jersyT'] = play_scout['jersyT'].astype(np.float)
play_scout['jersyG'] = play_scout['jersyG'].astype(np.float)
play_scout['jersyPR'] = play_scout['jersyPR'].astype(np.float)
play_scout['jersySTS'] = play_scout['jersySTS'].astype(np.float)
play_scout['jersyV'] = play_scout['jersyV'].astype(np.float)


#### Merging Play scout and Track dataframe

In [ ]:
#merging play scout and track based on Missed Tackler Player
play_scout = pd.merge(play_scout, track, how='left',left_on=['jersyMT','gameId','playId','missedTackler'],right_on=['jerseyNumber','gameId','playId','teamName'])

#clean un used coulumn
del play_scout['jerseyNumber']
del play_scout['teamName']

play_scout = play_scout.rename(columns={"nflId": "MTId"})


In [ ]:
#merging play scout and track based on Assist Tackler Player
play_scout = pd.merge(play_scout, track, how='left',left_on=['jersyAT','gameId','playId','assistTackler'],right_on=['jerseyNumber','gameId','playId','teamName'])
#clean un used coulumn
del play_scout['jerseyNumber']
del play_scout['teamName']
play_scout = play_scout.rename(columns={"nflId": "ATId"})


In [ ]:
#merging play scout and track based on  Tackler Player
play_scout = pd.merge(play_scout, track, how='left',left_on=['jersyT','gameId','playId','tackler'],right_on=['jerseyNumber','gameId','playId','teamName'])
#clean un used coulumn
del play_scout['jerseyNumber']
del play_scout['teamName']
play_scout = play_scout.rename(columns={"nflId": "TId"})



In [ ]:
#merging play scout and track based on  Gunners Player
play_scout = pd.merge(play_scout, track, how='left',left_on=['jersyG','gameId','playId','gunners'],right_on=['jerseyNumber','gameId','playId','teamName'])
del play_scout['jerseyNumber']
del play_scout['teamName']
play_scout = play_scout.rename(columns={"nflId": "GId"})

In [ ]:
#merging play scout and track based on  Punt Rusher Player
play_scout = pd.merge(play_scout, track, how='left',left_on=['jersyPR','gameId','playId','puntRushers'],right_on=['jerseyNumber','gameId','playId','teamName'])
del play_scout['jerseyNumber']
del play_scout['teamName']
play_scout = play_scout.rename(columns={"nflId": "PRId"})

In [ ]:
#merging play scout and track based on  specialTeamsSafeties Player
play_scout = pd.merge(play_scout, track, how='left',left_on=['jersySTS','gameId','playId','specialTeamsSafeties'],right_on=['jerseyNumber','gameId','playId','teamName'])
del play_scout['jerseyNumber']
del play_scout['teamName']
play_scout = play_scout.rename(columns={"nflId": "STSId"})



In [ ]:
#merging play scout and track based on  vises Player
play_scout = pd.merge(play_scout, track, how='left',left_on=['jersyV','gameId','playId','vises'],right_on=['jerseyNumber','gameId','playId','teamName'])
del play_scout['jerseyNumber']
del play_scout['teamName']
play_scout = play_scout.rename(columns={"nflId": "VId"})

In [ ]:
play_scout

### 5. Merge player and play_scout to define College name

In [ ]:
#merging play_scout and player 
play_scout = pd.merge(play_scout, players, how='left', left_on=['MTId'],right_on=['nflId'])
del play_scout['nflId']
play_scout = play_scout.rename(columns={"collegeName": "MTCollege"})

In [ ]:
play_scout = pd.merge(play_scout, players, how='left', left_on=['ATId'],right_on=['nflId'])
del play_scout['nflId']
play_scout = play_scout.rename(columns={"collegeName": "ATCollege"})

In [ ]:
play_scout = pd.merge(play_scout, players, how='left', left_on=['TId'],right_on=['nflId'])
del play_scout['nflId']
play_scout = play_scout.rename(columns={"collegeName": "TCollege"})

In [ ]:
play_scout = pd.merge(play_scout, players, how='left', left_on=['GId'],right_on=['nflId'])
del play_scout['nflId']
play_scout = play_scout.rename(columns={"collegeName": "GCollege"})

In [ ]:
play_scout = pd.merge(play_scout, players, how='left', left_on=['PRId'],right_on=['nflId'])
del play_scout['nflId']
play_scout = play_scout.rename(columns={"collegeName": "PRCollege"})

In [ ]:
play_scout = pd.merge(play_scout, players, how='left', left_on=['VId'],right_on=['nflId'])
del play_scout['nflId']
play_scout = play_scout.rename(columns={"collegeName": "VCollege"})

In [ ]:
play_scout.head()

### 6. Encoding

In [ ]:
# Import label encoder
from sklearn import preprocessing
 
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
play_scout['specialTeamsPlayType_cat']= label_encoder.fit_transform(play_scout['specialTeamsPlayType'])
play_scout['specialTeamsResult_cat']= label_encoder.fit_transform(play_scout['specialTeamsResult'])
play_scout['passResult_cat']= label_encoder.fit_transform(play_scout['passResult'].astype(str))
play_scout['kickType_cat']= label_encoder.fit_transform(play_scout['kickType'].astype(str))
play_scout['kickContactType_cat']= label_encoder.fit_transform(play_scout['kickContactType'].astype(str))
play_scout['kickerCollege_cat']= label_encoder.fit_transform(play_scout['kickerCollege'].astype(str))
play_scout['returnerCollege_cat']= label_encoder.fit_transform(play_scout['returnerCollege'].astype(str))

In [ ]:
play_scout.head()

## **BUILD MODEL & MATRIX EVALUATION**

## **Feature and Target Selection**

After the pre-processing stage is complete, the selected feature will be used for data mining and modelling algorithm implementation.

In [ ]:
pd.set_option('display.max_columns', None)
play_scout.head()

In [ ]:
dataModel = play_scout[['specialTeamsPlayType_cat','kickType_cat' , 'kickContactType_cat', 'kickAccuracy', 'returnAccuracy', 'kickLength', 'hangTime','playResult','eventScore','specialTeamsResult_cat']]

In [ ]:
dataModel = dataModel.dropna()

In [ ]:
dataModel.isnull().sum()

In [ ]:
#Feature and Target Selection
feature = dataModel[['specialTeamsPlayType_cat','kickType_cat' , 'kickContactType_cat', 'kickAccuracy', 'returnAccuracy', 'kickLength', 'hangTime','playResult']]
target = dataModel[['eventScore']]

## **Modelling and Evaluation Matrix**

1. We do the data mining techniques in this step. This process aims to extract potential patterns to obtain useful data that will be used to solve existing problems. We use traditional methods, namely Decission Tree, Random Forest, KNN, Linear Regression, SGD, SVC and advanced algorithms, namely SVM,MLP, and CNN. It aims to compare the method used to determine the toughness of the algorithm being tested.
2. At this evaluation part, the results of data mining techniques in the form of typical patterns and predictive models are evaluated to assess whether the existing hypothesis is indeed achieved. We will use Evaluation metrics such as recall, f-1 score, and precision.

I purposely run data modeling individually for each algorithm so that the data splitting process is also different and the data is randomized as a whole for each model

## 1. KNN

In [ ]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, shuffle=True) # 70% training and 30% test

In [ ]:
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier

#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5)

#Train the model using the training sets
knn.fit(X_train, y_train.values.ravel())

#Predict the response for test dataset
y_pred = knn.predict(X_test)

#Predict the response for test dataset
x_pred = knn.predict(X_train)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

#Confusion Matrix for Training Data
print(confusion_matrix(y_train,x_pred))
print(classification_report(y_train,x_pred))

In [ ]:
#Confusion Matrix for Testing Data
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

## 2. RF

In [ ]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, shuffle=True) # 70% training and 30% test

In [ ]:
#Import knearest neighbors Classifier model
from sklearn.ensemble import RandomForestClassifier

#Create KNN Classifier
rf = RandomForestClassifier()

#Train the model using the training sets
rf.fit(X_train, y_train.values.ravel())

#Predict the response for test dataset
y_pred = rf.predict(X_test)

x_pred =rf.predict(X_train)

In [ ]:
#Confusion Matrix for Training Data
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,x_pred))
print(classification_report(y_train,x_pred))

In [ ]:
#Confusion Matrix for Testing Data
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

## 3. Decission Tree

In [ ]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, shuffle=True) # 70% training and 30% test

In [ ]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()

#Train the model using the training sets
DT.fit(X_train, y_train.values.ravel())

#Predict the response for test dataset
y_pred = DT.predict(X_test)

x_pred = DT.predict(X_train)

In [ ]:
#Confusion Matrix for Training Data
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,x_pred))
print(classification_report(y_train,x_pred))

In [ ]:
#Confusion Matrix for Testing Data
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

## 4. SVC

In [ ]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, shuffle=True) # 70% training and 30% test

In [ ]:
#Import Logistic SVC model
from sklearn.svm import SVC, LinearSVC

#Create KNN Classifier
SVC = SVC()

#Train the model using the training sets
SVC.fit(X_train, y_train.values.ravel())

#Predict the response for test dataset
y_pred = SVC.predict(X_test)

x_pred =SVC.predict(X_train)

In [ ]:
#Confusion Matrix for Training Data
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,x_pred))
print(classification_report(y_train,x_pred))

In [ ]:
#Confusion Matrix for Testing Data
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

## 5. Linier SVC

In [ ]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, shuffle=True) # 70% training and 30% test

In [ ]:
#Import Logistic SVC model
from sklearn.svm import SVC, LinearSVC

#Create KNN Classifier
LSVC = LinearSVC()

#Train the model using the training sets
LSVC.fit(X_train, y_train.values.ravel())

#Predict the response for test dataset
y_pred = LSVC.predict(X_test)

x_pred = LSVC.predict(X_train)

In [ ]:
#Confusion Matrix for Training Data
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,x_pred))
print(classification_report(y_train,x_pred))

In [ ]:
#Confusion Matrix for Testing Data
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

## 6. SGD

In [ ]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, shuffle=True) # 70% training and 30% test

In [ ]:
from sklearn.linear_model import LogisticRegression, Ridge, SGDClassifier

SGD = SGDClassifier()

#Train the model using the training sets
SGD.fit(X_train, y_train.values.ravel())

#Predict the response for test dataset
y_pred = SGD.predict(X_test)

x_pred =SGD.predict(X_train)

In [ ]:
#Confusion Matrix for Training Data
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,x_pred))
print(classification_report(y_train,x_pred))

In [ ]:
#Confusion Matrix for Testing Data
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

## 7. MLP

In [ ]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, shuffle=True) # 70% training and 30% test

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500)
mlp.fit(X_train,y_train)

predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)

In [ ]:
#Confusion Matrix for Training Data
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))

In [ ]:
#Confusion Matrix for Testing Data
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))

## 8. CNN

In [ ]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, shuffle=True) # 70% training and 30% test

In [ ]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

# Initialising the ANN
classifier = Sequential()

In [ ]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim =8))

# Adding the second hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 10)

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

x_pred = classifier.predict(X_train)
x_pred = (x_pred > 0.5)

In [ ]:
#Confusion Matrix for Training Data
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,x_pred))
print(classification_report(y_train,x_pred))

In [ ]:
#Confusion Matrix for Testing Data
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

## 10. SVM

In [ ]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2) # 70% training and 30% test

In [ ]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

x_pred = clf.predict(X_train)

In [ ]:
#Confusion Matrix for Training Data
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,x_pred))
print(classification_report(y_train,x_pred))

In [ ]:
#Confusion Matrix for Testing Data
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

# **PART II. KNOWLEDGE GRAPH**

### **DATA PREPARATION**

In [ ]:
dfgraph= play_scout[['kickerId', 'returnerId', 'specialTeamsResult', 'kickerCollege','returnerCollege', 'eventScore', 'MTId', 'ATId', 'TId', 'GId', 'PRId', 'STSId', 'VId', 'MTCollege', 'ATCollege', 'TCollege', 'GCollege', 'PRCollege', 'STSCollege', 'VCollege']] 
dfgraph.head()

## 1. Kicker Returner Relation

In [ ]:
dfgraphKR= dfgraph[['kickerId', 'returnerId', 'specialTeamsResult', 'kickerCollege','returnerCollege', 'eventScore']] 

In [ ]:
dfgraphKR.isnull().sum()

In [ ]:
dfgraphKR.dropna(subset = ["kickerId"], inplace=True)
dfgraphKR.dropna(subset = ["returnerId"], inplace=True)
dfgraphKR.dropna(subset = ["kickerCollege"], inplace=True)

In [ ]:
dfgraphKR = dfgraphKR.reset_index(drop=True)

In [ ]:
#define team name for each player based on home team and visitor team
for x in range(len(dfgraphKR)-1):
  if dfgraphKR.loc[x, 'kickerCollege'] == dfgraphKR.loc[x, 'returnerCollege']:
    dfgraphKR.loc[x, 'KRCollege'] = 1
  else:
    dfgraphKR.loc[x, 'KRCollege'] = 0

In [ ]:
dfgraphKR.head()

In [ ]:
dfgraphKR['kickerId'] =dfgraphKR['kickerId'].astype(int)

#### Graph Construction and Visualization

In [ ]:
GDirectKR = nx.from_pandas_edgelist(dfgraphKR,source='kickerId',target='returnerId', create_using=nx.DiGraph(),edge_attr='specialTeamsResult')

In [ ]:
degCent = nx.degree_centrality(GDirectKR)
node_color = ['orange' if type(v)==float else 'green' for v in GDirectKR]
node_size =  [v * 10000 for v in degCent.values()]
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GDirectKR, with_labels=True,
                 node_color=node_color,
                 node_size=node_size)

plt.show()

#### College Relation

In [ ]:
#college team

GCollegeKR = nx.from_pandas_edgelist(dfgraphKR,source='kickerCollege',target='returnerCollege', create_using=nx.DiGraph(),edge_attr='KRCollege')

In [ ]:
color = [
"lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy"]

In [ ]:
colors = []
i = 0
for node in GCollegeKR:
    if node in dfgraphKR["kickerId"].values:
      colors.append("navy")  
    elif node in dfgraphKR["returnerId"].values:
      colors.append("lightgreen")
    else:
        colors.append(color[i])
        i+=1

In [ ]:
degCent = nx.degree_centrality(GCollegeKR)
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GCollegeKR, with_labels=True,
                 node_color=colors,
                 node_size=1200)

plt.show()

![Kicker And Returner Relationship With Others Player Type](https://drive.google.com/file/d/1bn9eT9lHND8gqGVwm6PA3fBbN1PuRTaE/view?usp=sharing)



## 2. Relation between Kicker and Other Player Type

### 2.1 Kicker and Gunners Relation

In [ ]:
dfgraphKG= dfgraph[['kickerId', 'GId', 'specialTeamsResult', 'kickerCollege','GCollege', 'eventScore']] 

In [ ]:
dfgraphKG.isnull().sum()

In [ ]:
dfgraphKG.dropna(subset = ["kickerId"], inplace=True)
dfgraphKG.dropna(subset = ["GId"], inplace=True)
dfgraphKG.dropna(subset = ["GCollege"], inplace=True)

In [ ]:
dfgraphKG['kickerId'] =dfgraphKG['kickerId'].astype(int)

#### Graph Construction and Visualization

In [ ]:
GDirectKG = nx.from_pandas_edgelist(dfgraphKG,source='kickerId',target='GId', create_using=nx.DiGraph(),edge_attr='specialTeamsResult')

In [ ]:
degCent = nx.degree_centrality(GDirectKG)
node_color = ['orange' if type(v)==float else 'green' for v in GDirectKG]
node_size =  [v * 10000 for v in degCent.values()]
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GDirectKG, with_labels=True,
                 node_color=node_color,
                 node_size=node_size)

plt.show()

#### College Relation

In [ ]:
#college team

GCollegeKG = nx.from_pandas_edgelist(dfgraphKG,source='kickerCollege',target='GCollege', create_using=nx.DiGraph(),edge_attr='eventScore')

color = [
"lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy"]

colors = []
i = 0
for node in GCollegeKG:
    if node in dfgraphKG["kickerId"].values:
      colors.append("navy")  
    elif node in dfgraphKG["GId"].values:
      colors.append("lightgreen")
    else:
        colors.append(color[i])
        i+=1

        
#ploting graph
degCent = nx.degree_centrality(GCollegeKG)
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GCollegeKG, with_labels=True,
                 node_color=colors,
                 node_size=1200)

plt.show()

### 2.2. Kicker and Tackler Relation

In [ ]:
dfgraphKT= dfgraph[['kickerId', 'TId', 'specialTeamsResult', 'kickerCollege','TCollege', 'eventScore']] 

In [ ]:
dfgraphKT.isnull().sum()

In [ ]:
dfgraphKT.dropna(subset = ["kickerId"], inplace=True)
dfgraphKT.dropna(subset = ["TId"], inplace=True)
dfgraphKT.dropna(subset = ["TCollege"], inplace=True)

In [ ]:
dfgraphKT['kickerId'] =dfgraphKT['kickerId'].astype(int)

#### Graph Construction and Visualization

In [ ]:
GDirectKT = nx.from_pandas_edgelist(dfgraphKT,source='kickerId',target='TId', create_using=nx.DiGraph(),edge_attr='specialTeamsResult')

degCent = nx.degree_centrality(GDirectKT)
node_color = ['orange' if type(v)==float else 'green' for v in GDirectKT]
node_size =  [v * 10000 for v in degCent.values()]
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GDirectKT, with_labels=True,
                 node_color=node_color,
                 node_size=node_size)

plt.show()

#### College Relation

In [ ]:
#college team

GCollegeKT = nx.from_pandas_edgelist(dfgraphKT,source='kickerCollege',target='TCollege', create_using=nx.DiGraph(),edge_attr='eventScore')

color = [
"lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy"]

colors = []
i = 0
for node in GCollegeKT:
    if node in dfgraphKT["kickerId"].values:
      colors.append("navy")  
    elif node in dfgraphKT["TId"].values:
      colors.append("lightgreen")
    else:
        colors.append(color[i])
        i+=1

        
#ploting graph
degCent = nx.degree_centrality(GCollegeKT)
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GCollegeKT, with_labels=True,
                 node_color=colors,
                 node_size=1200)

plt.show()

### 2.3 Kicker and Asist Tackler Relation 

In [ ]:
dfgraphKAT= dfgraph[['kickerId', 'ATId', 'specialTeamsResult', 'kickerCollege','ATCollege', 'eventScore']] 

In [ ]:
dfgraphKAT.isnull().sum()

In [ ]:
dfgraphKAT.dropna(subset = ["kickerId"], inplace=True)
dfgraphKAT.dropna(subset = ["ATId"], inplace=True)
dfgraphKAT.dropna(subset = ["ATCollege"], inplace=True)

In [ ]:
dfgraphKAT['kickerId'] =dfgraphKAT['kickerId'].astype(int)

#### Graph Construction and Visualization

In [ ]:
GDirectKAT = nx.from_pandas_edgelist(dfgraphKAT,source='kickerId',target='ATId', create_using=nx.DiGraph(),edge_attr='specialTeamsResult')

degCent = nx.degree_centrality(GDirectKAT)
node_color = ['orange' if type(v)==float else 'green' for v in GDirectKAT]
node_size =  [v * 10000 for v in degCent.values()]
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GDirectKAT, with_labels=True,
                 node_color=node_color,
                 node_size=node_size)

plt.show()

#### College Relation Between Kicker and Assist Tackler

In [ ]:
#college team

GCollegeKAT = nx.from_pandas_edgelist(dfgraphKAT,source='kickerCollege',target='ATCollege', create_using=nx.DiGraph(),edge_attr='eventScore')

color = [
"lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy"]

colors = []
i = 0
for node in GCollegeKAT:
    if node in dfgraphKAT["kickerId"].values:
      colors.append("navy")  
    elif node in dfgraphKAT["ATId"].values:
      colors.append("lightgreen")
    else:
        colors.append(color[i])
        i+=1

        
#ploting graph
degCent = nx.degree_centrality(GCollegeKAT)
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GCollegeKAT, with_labels=True,
                 node_color=colors,
                 node_size=1200)

plt.show()

## 3. Relation between Retuner and Other Player Type

### 3.1 Returner and Punt Rusher Relation

In [ ]:
dfgraphRPR= dfgraph[['returnerId', 'PRId', 'specialTeamsResult', 'returnerCollege','PRCollege', 'eventScore']] 

In [ ]:
dfgraphRPR.isnull().sum()

In [ ]:
dfgraphRPR.dropna(subset = ["returnerId"], inplace=True)
dfgraphRPR.dropna(subset = ["PRId"], inplace=True)
dfgraphRPR.dropna(subset = ["PRCollege"], inplace=True)

In [ ]:
dfgraphRPR['returnerId'] =dfgraphRPR['returnerId'].astype(int)

#### Graph Construction and Visualization

In [ ]:
GDirectRPR = nx.from_pandas_edgelist(dfgraphRPR,source='returnerId',target='PRId', create_using=nx.DiGraph(),edge_attr='specialTeamsResult')

degCent = nx.degree_centrality(GDirectRPR)
node_color = ['red' if type(v)==float else 'blue' for v in GDirectRPR]
node_size =  [v * 10000 for v in degCent.values()]
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GDirectRPR, with_labels=True,
                 node_color=node_color,
                 node_size=node_size)

plt.show()

#### College Relation Between Returner and Punt Rusher

In [ ]:
#college team

GCollegeRPR = nx.from_pandas_edgelist(dfgraphRPR,source='returnerCollege',target='PRCollege', create_using=nx.DiGraph(),edge_attr='eventScore')

color = [
"lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy"]

colors = []
i = 0
for node in GCollegeRPR:
    if node in dfgraphRPR["returnerId"].values:
      colors.append("navy")  
    elif node in dfgraphRPR["PRId"].values:
      colors.append("lightgreen")
    else:
        colors.append(color[i])
        i+=1

        
#ploting graph
degCent = nx.degree_centrality(GCollegeRPR)
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GCollegeRPR, with_labels=True,
                 node_color=colors,
                 node_size=1200)

plt.show()

### 3.2 Returner and STS Relation

In [ ]:
dfgraphRSTS= dfgraph[['returnerId', 'STSId', 'specialTeamsResult', 'returnerCollege','STSCollege', 'eventScore']] 

In [ ]:
dfgraphRSTS.isnull().sum()

In [ ]:
dfgraphRSTS.dropna(subset = ["returnerId"], inplace=True)
dfgraphRSTS.dropna(subset = ["STSId"], inplace=True)
dfgraphRSTS.dropna(subset = ["STSCollege"], inplace=True)

In [ ]:
dfgraphRSTS['returnerId'] =dfgraphRSTS['returnerId'].astype(int)

#### Graph Construction and Visualization

In [ ]:
GDirectRSTS = nx.from_pandas_edgelist(dfgraphRSTS,source='returnerId',target='STSId', create_using=nx.DiGraph(),edge_attr='specialTeamsResult')

degCent = nx.degree_centrality(GDirectRSTS)
node_color = ['red' if type(v)==float else 'blue' for v in GDirectRSTS]
node_size =  [v * 10000 for v in degCent.values()]
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GDirectRSTS, with_labels=True,
                 node_color=node_color,
                 node_size=node_size)

plt.show()

#### College Relation Between Returner and STS

In [ ]:
#college team

GCollegeRSTS = nx.from_pandas_edgelist(dfgraphRSTS,source='returnerCollege',target='STSCollege', create_using=nx.DiGraph(),edge_attr='eventScore')

color = [
"lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy"]

colors = []
i = 0
for node in GCollegeRSTS:
    if node in dfgraphRSTS["returnerId"].values:
      colors.append("navy")  
    elif node in dfgraphRSTS["STSId"].values:
      colors.append("lightgreen")
    else:
        colors.append(color[i])
        i+=1

        
#ploting graph
degCent = nx.degree_centrality(GCollegeRSTS)
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GCollegeRSTS, with_labels=True,
                 node_color=colors,
                 node_size=1200)

plt.show()

### 3.3 Returner and Vises Relation

In [ ]:
dfgraphRV= dfgraph[['returnerId', 'VId', 'specialTeamsResult', 'returnerCollege','VCollege', 'eventScore']] 

In [ ]:
dfgraphRV.isnull().sum()

In [ ]:
dfgraphRV.dropna(subset = ["returnerId"], inplace=True)
dfgraphRV.dropna(subset = ["VId"], inplace=True)
dfgraphRV.dropna(subset = ["VCollege"], inplace=True)

In [ ]:
dfgraphRV['returnerId'] =dfgraphRV['returnerId'].astype(int)

#### Graph Construction and Visualization

In [ ]:
GDirectRV = nx.from_pandas_edgelist(dfgraphRV,source='returnerId',target='VId', create_using=nx.DiGraph(),edge_attr='specialTeamsResult')

degCent = nx.degree_centrality(GDirectRV)
node_color = ['red' if type(v)==float else 'blue' for v in GDirectRV]
node_size =  [v * 10000 for v in degCent.values()]
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GDirectRV, with_labels=True,
                 node_color=node_color,
                 node_size=node_size)

plt.show()

#### College Relation Between Returner and Vises

In [ ]:
#college team

GCollegeRV = nx.from_pandas_edgelist(dfgraphRV,source='returnerCollege',target='VCollege', create_using=nx.DiGraph(),edge_attr='eventScore')

color = [
"lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy","lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy"]

colors = []
i = 0
for node in GCollegeRV:
    if node in dfgraphRV["returnerId"].values:
      colors.append("navy")  
    elif node in dfgraphRV["VId"].values:
      colors.append("lightgreen")
    else:
        colors.append(color[i])
        i+=1

        
#ploting graph
degCent = nx.degree_centrality(GCollegeRV)
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GCollegeRV, with_labels=True,
                 node_color=colors,
                 node_size=1200)

plt.show()

## GRAPH ANALYSIS

In [ ]:
returner_nodes = list(n for n in  GDirectRV.nodes() if type(n) == str)
len(returner_nodes)

In [ ]:
kicker_nodes = list(n for n in  GDirectRV.nodes() if type(n) == int)
len(kicker_nodes)

In [ ]:
from collections import defaultdict

def shared_partition_nodes(GDirectKR , node1, node2):
    # Check that the nodes belong to the same partition
    assert type(node1) == type(node2)

    # Get neighbors of node 1: nbrs1
    nbrs1 = GDirectKR.neighbors(node1)
    # Get neighbors of node 2: nbrs2
    nbrs2 = GDirectKR.neighbors(node2)

    # Compute the overlap using set intersections
    overlap = set(nbrs1).intersection(nbrs2)
    return overlap

def kick_similarity(GDirectKR , kick1, kick2, return_nodes):
    # Check that the nodes belong to the 'users' partition
    assert type(kick1) == int
    assert type(kick2) == int

    # Get the set of nodes shared between the two users
    shared_nodes = shared_partition_nodes(GDirectKR , kick1, kick2)

    # Return the fraction of nodes in the projects partition
    return len(shared_nodes) / len(return_nodes)



def kick_similar_users(GDirectKR , kick1, kick_nodes, return_nodes):
    # Data checks
    assert type(kick1) == int

    # Get other nodes from user partition
    kick_nodes = set(kick_nodes)
    kick_nodes.remove(kick1)

    # Create the dictionary: similarities
    similarities = defaultdict(list)
    for kick2 in kick_nodes:
        similarity = kick_similarity(GDirectKR , kick1, kick2, return_nodes)
        similarities[similarity].append(kick2)

    # Compute maximum similarity score: max_similarity
    max_similarity = max(similarities.keys())

    # Return list of users that share maximal similarity
    print(max_similarity)
    return similarities[max_similarity]

In [ ]:
print(kick_similar_users(GDirectRV, 44923, kicker_nodes, returner_nodes))

##STOP DISINI

In [ ]:
#count the number of special team for each play type
plays_specialTeams = plays.groupby('specialTeamsPlayType')
plays_specialTeams['specialTeamsPlayType'].count()

In [ ]:
#show only the important data related to task, and copy from original data frame

playsdata= plays[['gameId','kickerId','specialTeamsPlayType','specialTeamsResult','possessionTeam','returnerId']]
playsdata.head()

In [ ]:
playsdata.shape

In [ ]:
#grouping and counting data by PlayType, Teamresult, and returnerId

groupPlays = playsdata.groupby(['specialTeamsPlayType','specialTeamsResult'])['returnerId'].count()
groupPlays

In [ ]:
#check null value

playsdata.isnull().sum()

In [ ]:
#drop null value
playsdata= playsdata.dropna()
playsdata

In [ ]:
playsdata.isnull().sum()

In [ ]:
#check duplicate data
playsData = playsdata.groupby(['gameId','possessionTeam', 'specialTeamsPlayType','specialTeamsResult','kickerId','returnerId'])
playsData['returnerId'].count()

In [ ]:
#drop duplicate data
playsdata= playsdata.drop_duplicates()
playsdata

In [ ]:
playsdata.shape

# Graph Construction and Visualization

## Undirect graph

An undirected chart is a more restrictive chart type. They only represent whether there is a relationship between two nodes or not. However they do not represent the difference between subject and object in that relationship. to represent between kicker and returner for the whole data, it is better to use indirect without considering the relation between kicker and receiver. If using a direct graph, the relationship must be built by looking at the data per each team.

In [ ]:
# Add all columns in the table as edge attribute
G = nx.from_pandas_edgelist(playsdata, 'kickerId', 'returnerId', edge_attr=True )

In [ ]:
#check the edges
G.edges(data=True)

In [ ]:
#check the nodes
G.nodes()

In [ ]:
print(nx.info(G))

In [ ]:
#check the type of Graph
type(G)

In [ ]:
#check data type of playsdata frame
playsdata.dtypes

Print out an undirected graph with all the kicker-and-returner graphs.

In [ ]:
figure(figsize=(20, 20))
nx.draw_networkx(G, with_labels = True)

In [ ]:
#print graph using degree of centrality

# set layout (node position)
pos = nx.spring_layout(G)

betCent = nx.betweenness_centrality(G, normalized=True, endpoints=True)
node_color = [20000.0 * G.degree(v) for v in G]
node_size =  [v * 10000 for v in betCent.values()]
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(G, pos=pos, with_labels=False,
                 node_color=node_color,
                 node_size=node_size )

plt.show()
# may took some times

## Direct Graph for each team

Construct Direct Graph for each team

In [ ]:
#change datatype of kicker ID from float to int
playsdata['kickerId'] =playsdata['kickerId'].astype(int)

In [ ]:
playsdata.dtypes

In [ ]:
playsdataGroup = playsdata.groupby('possessionTeam')

In [ ]:
keys = playsdataGroup.groups.keys()

for i in keys:
    G_Data = nx.from_pandas_edgelist(playsdataGroup.get_group(i),source='kickerId',target='returnerId', create_using=nx.DiGraph(),edge_attr=True)

### Print out the graph (with larger size than default) for Team "DET" with player names on nodes

In [ ]:
playsdataDET=playsdataGroup.get_group("DET")

In [ ]:
#construct teams DET direct graph
GDET = nx.from_pandas_edgelist(playsdataDET,source='kickerId',target='returnerId', create_using=nx.DiGraph(),edge_attr='specialTeamsPlayType')

In [ ]:
#print Graph of team DET

degCent = nx.degree_centrality(GDET)
node_color = ['orange' if type(v)==str else 'green' for v in GDET]
node_size =  [v * 10000 for v in degCent.values()]
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GDET, with_labels=True,
                 node_color=node_color,
                 node_size=node_size)

plt.show()

## Bipartie Graph

The vertices in the graph can be divided into two independent sets. in this case the set is U and V with colors, green and orange. We can also say that there is no edge that connects vertices of same set. A bipartite graph is possible if the graph coloring is possible using two colors such that vertices in a set are colored with the same color.

In [ ]:
for node in GDET.nodes:
    if type(node) == str:
        GDET.nodes[node]['bipartie']='kickerId'
    elif type(node) == int:
        GDET.nodes[node]['bipartie']='returnerId'

In [ ]:
top = {node for node in GDET.nodes if GDET.nodes[node]['bipartie']=='kickerId'}

In [ ]:
pos = nx.bipartite_layout(GDET, top)

degCent = nx.degree_centrality(GDET)
node_color = ['orange' if type(v)==str else 'green' for v in GDET]
node_size =  [v * 10000 for v in degCent.values()]
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GDET, pos=pos, with_labels=True,
                 node_color=node_color,
                 node_size=node_size )

plt.show()

## Extend the graph for Teams "ATL" with each returner connected to its corresponding Team

In [ ]:
#copy only specific coulumn that will be use in task

gamesdata = games[['gameId','homeTeamAbbr','visitorTeamAbbr']]
gamesdata.head()

In [ ]:
gamesdata.shape

In [ ]:
#add new coulumn name of returner team into first dataframe by gameId
playsdata['homeTeam'] =playsdata.gameId.map(gamesdata.set_index('gameId')['homeTeamAbbr'].to_dict())

In [ ]:
playsdata

In [ ]:
playsdata2=playsdata.merge(games[['gameId','homeTeamAbbr','visitorTeamAbbr']],how='left',left_on=['gameId','possessionTeam'],right_on=['gameId','homeTeamAbbr']).drop('homeTeamAbbr', axis='columns').fillna(games['homeTeamAbbr'])

In [ ]:
for i in range(len(playsdata2)):
  if (playsdata2['possessionTeam'][i] != playsdata2['homeTeam'][i]) :
     playsdata2['visitorTeamAbbr'][i] = playsdata2['homeTeam'][i]

In [ ]:
playsdata2 = playsdata2.rename(columns={"visitorTeamAbbr": "returnerTeam"})

In [ ]:
playsdata2

In [ ]:
playsdata2Group = playsdata2.groupby('possessionTeam')

In [ ]:
dataATL=playsdata2Group.get_group("ATL")

In [ ]:
GATL = nx.from_pandas_edgelist(dataATL,source='returnerId',target='returnerTeam', create_using=nx.DiGraph(),edge_attr='possessionTeam')

In [ ]:
len(pd.unique(dataATL["returnerTeam"]))

In [ ]:
color = [
"lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "purple", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue","orange","green","navy"]

In [ ]:
colors = []
i = 0
for node in GATL:
    if node in dataATL["returnerId"].values:
      colors.append("lightgreen")  
    else:
        colors.append(color[i])
        i+=1


In [ ]:
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GATL, with_labels=True,
                 node_color=colors,
                 node_size=1200)

plt.show()

## Graph Analysis

In thhe directed graph of Team "DET", we want to know which player acts the most similar functionality as player 40113. We can calculate the similarity by finding the kick of similiarity between the nodes of each player. below is a code of displaying the similiarity of players who have similiarity to certain players. From the results we can also see which players have the same function as the player number 40113 . 

In [ ]:
returner_nodes = list(n for n in  GDET.nodes() if type(n) == str)
len(returner_nodes)

In [ ]:
kicker_nodes = list(n for n in  GDET.nodes() if type(n) == int)
len(kicker_nodes)

In [ ]:
from collections import defaultdict

def shared_partition_nodes(G, node1, node2):
    # Check that the nodes belong to the same partition
    assert type(node1) == type(node2)

    # Get neighbors of node 1: nbrs1
    nbrs1 = G.neighbors(node1)
    # Get neighbors of node 2: nbrs2
    nbrs2 = G.neighbors(node2)

    # Compute the overlap using set intersections
    overlap = set(nbrs1).intersection(nbrs2)
    return overlap

def kick_similarity(G, kick1, kick2, return_nodes):
    # Check that the nodes belong to the 'users' partition
    assert type(kick1) == int
    assert type(kick2) == int

    # Get the set of nodes shared between the two users
    shared_nodes = shared_partition_nodes(G, kick1, kick2)

    # Return the fraction of nodes in the projects partition
    return len(shared_nodes) / len(return_nodes)



def kick_similar_users(G, kick1, kick_nodes, return_nodes):
    # Data checks
    assert type(kick1) == int

    # Get other nodes from user partition
    kick_nodes = set(kick_nodes)
    kick_nodes.remove(kick1)

    # Create the dictionary: similarities
    similarities = defaultdict(list)
    for kick2 in kick_nodes:
        similarity = kick_similarity(G, kick1, kick2, return_nodes)
        similarities[similarity].append(kick2)

    # Compute maximum similarity score: max_similarity
    max_similarity = max(similarities.keys())

    # Return list of users that share maximal similarity
    print(max_similarity)
    return similarities[max_similarity]

In [ ]:
print(kick_similar_users(GDET, 40113, kicker_nodes, returner_nodes))

In the directed graph of Team "ATL", we want to know which player is the most important player in the Team from graph's view. From graph view we can see the most important player in ATL teams is 37267. We can calculate it using degree of centrality methods. The most important player is the most player that give action in games wheter as kicker or as a receiver.

In [ ]:
dataATL=playsdataGroup.get_group("ATL")
GATL = nx.from_pandas_edgelist(dataATL,source='kickerId',target='returnerId', create_using=nx.DiGraph(),edge_attr='specialTeamsPlayType')

In [ ]:
node_to_neighbors_mapping = [(node, len(list(GATL.neighbors(node)))) for node in GATL.nodes()]
node_to_neighbors_mapping

In [ ]:
node_to_neighbors_ser = pd.Series(data=dict(node_to_neighbors_mapping))

node_to_neighbors_ser.sort_values(ascending=False).head()

In [ ]:
degCent = nx.degree_centrality(GATL)
node_color = ['orange' if type(v)==str else 'green' for v in GATL]
node_size =  [v * 10000 for v in degCent.values()]
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(GATL, with_labels=True,
                 node_color=node_color,
                 node_size=node_size)

plt.show()

### Now, we want to construct the graph that representing the Special Teams Play Type with GameId so we can find the most Play Type that have been using in NFL American Football By Special Teams


In [ ]:
#now, we want to construct the graph that representing the Special Teams Play Type with GameId so we can find the most Play Type that have been using in NFL American Football By Special Teams

# Add all columns in the table as edge attribute
G = nx.from_pandas_edgelist(playsdata, 'gameId', 'specialTeamsPlayType', edge_attr=True )

In [ ]:
# Add all columns in the table as edge attribute
G2 = nx.from_pandas_edgelist(playsdata, 'gameId', 'specialTeamsResult', edge_attr=True )

In [ ]:
degCent = nx.degree_centrality(G)
node_color = ['orange' if type(v)==int else 'green' for v in G]
node_size =  [v * 10000 for v in degCent.values()]
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(G, with_labels=True,
                 node_color=node_color,
                 node_size=node_size)

plt.show()


#From the graph below  we can see that the most Play type that have been using by Special Team is "Kickoff"

### Besides, we can also construct the graph that representing the Special Teams Result with GameId so we can find the most Special Teams strategie resulted in NFL American Football we can achieve it using degree of centrality


In [ ]:

degCent = nx.degree_centrality(G2)
node_color = ['blue' if type(v)==int else 'red' for v in G2]
node_size =  [v * 10000 for v in degCent.values()]
plt.figure(figsize=(20,20))

# draw figure with customized options
nx.draw(G2, with_labels=True,
                 node_color=node_color,
                 node_size=node_size)

plt.show()


#From the graph below  we can see that the most Result is "Return"

To be continous....